In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
mpl.rc("figure", dpi=50)

from ubergauss import tools as t
import numpy as np
path = '/home/ubuntu/data/scdata/'
from cellsaw import load as loader
ds = [
 #'Bonemarrow_10xchromium_SRA779509-SRS3805245_6248',
 'Bonemarrow_10xchromium_SRA779509-SRS3805246_6024',
 'Bonemarrow_10xchromium_SRA779509-SRS3805247_7623',
 'Bonemarrow_10xchromium_SRA779509-SRS3805248_6395',
 'Bonemarrow_10xchromium_SRA779509-SRS3805255_7559',
 'Bonemarrow_10xchromium_SRA779509-SRS3805258_5683',
 'Bonemarrow_10xchromium_SRA779509-SRS3805262_6431',
 'Bonemarrow_10xchromium_SRA779509-SRS3805266_6210',
 'Colon_10xchromium_SRA703206-SRS3296611_4826',
 'Colon_10xchromium_SRA703206-SRS3296612_6476',
 'Colon_10xchromium_SRA728025-SRS3454422_2322',
 'Colon_10xchromium_SRA728025-SRS3454423_2283',
 'Colon_10xchromium_SRA728025-SRS3454424_1898',
 'Colon_10xchromium_SRA728025-SRS3454425_3429',
 'Colon_10xchromium_SRA728025-SRS3454426_3423',
 'Colon_10xchromium_SRA728025-SRS3454427_3840',
 'Colon_10xchromium_SRA728025-SRS3454428_5459',
 'Colon_10xchromium_SRA728025-SRS3454430_4910',
 'Liver_10xchromium_SRA716608-SRS3391629_4190',
 #'Liver_10xchromium_SRA716608-SRS3391630_4719',
 'Liver_10xchromium_SRA716608-SRS3391631_1425',
 'Liver_10xchromium_SRA716608-SRS3391632_6158',
 'Liver_10xchromium_SRA716608-SRS3391633_6806',
 'Placenta_10xchromium_SRA782908-SRS3815600_7205',
 'Prostate_10xchromium_SRA742961-SRS3565195_6101',
 'Prostate_10xchromium_SRA742961-SRS3565196_7789',
 'Prostate_10xchromium_SRA742961-SRS3565197_11468',
 'Prostate_10xchromium_SRA742961-SRS3565199_8069',
 'Prostate_10xchromium_SRA742961-SRS3565201_7479',
 'Prostate_10xchromium_SRA742961-SRS3565203_6453',
 'Prostate_10xchromium_SRA742961-SRS3565211_6264',
 'Tcells_10xchromium_SRA665712-SRS3034950_10979',
 'Tcells_10xchromium_SRA665712-SRS3034951_14332',
 'Tcells_10xchromium_SRA665712-SRS3034953_12974',
 'Tcells_10xchromium_SRA794656-SRS3937924_1878',
 'Tcells_10xchromium_SRA794656-SRS3937926_1289',
 'Tcells_10xchromium_SRA814476-SRS4073850_1975',
 'Tcells_drop-seq_SRA867342-SRS4550172_4068',
 'Tcells_drop-seq_SRA867342-SRS4550173_2768',
 'Testicle_10xchromium_SRA667709-SRS3065426_2777',
 'Testicle_10xchromium_SRA667709-SRS3065427_3045',
 'Testicle_10xchromium_SRA667709-SRS3065428_3007',
 'Testicle_10xchromium_SRA667709-SRS3065429_3066',
 'Testicle_10xchromium_SRA667709-SRS3065430_4020',
 'Testicle_10xchromium_SRA667709-SRS3065431_3586',
 'Testis_10xchromium_SRA645804-SRS2823404_4197',
 'Testis_10xchromium_SRA645804-SRS2823405_3598',
 'Testis_10xchromium_SRA645804-SRS2823406_3989',
 'Testis_10xchromium_SRA645804-SRS2823407_4046',
 'Testis_10xchromium_SRA645804-SRS2823408_4306',
 'Testis_10xchromium_SRA645804-SRS2823409_4791',
 'Testis_10xchromium_SRA645804-SRS2823411_2167',
 'Testis_10xchromium_SRA645804-SRS2823412_5299',
 'Testis_10xchromium_SRA645804-SRS3572594_4574']

In [ ]:
%%time

'''
this reads from csv files and saves as h5 to make reading faster in the future.
it also tries to add 'tissue' 'tissue5' and 'tissue5id' fields to 'uns'. 
'''
# TODO reader writes obs['barcode'] = index to the adata
datasets = loader.read(dir = '/home/ubuntu/data/scdata',
                       datasets=ds, 
                       suffix = '.counts.gz')


In [ ]:
%%time
dnames = ds
datasets = loader.read(dir = path,datasets = dnames, suffix = '.h5')



datasets = loader.annotate(datasets,
                          annotate_function = loader.annotate_from_barcode_csv,
                          input_field='barcode',
                          output_field = 'clusterid')


datasets = loader.annotate(datasets, annotate_function= loader.annotate_celltypes,
                          input_field='clusterid',
                          output_field = 'celltype',
                          path= path)



loader.save(datasets, format = 'h5')


In [ ]:
'''
%%time
# so we do stuff in chunks because memory
for dnames in t.Grouper(ds,10):
    #dnames = [d for d in dnames if d] # mm i need to disable nonpad
    #print(dnames)
    datasets = loader.read(dir = path,datasets = dnames, suffix = '.h5')
    #print([d.uns.get('fname',-1) for d in datasets])
    clusteranno  = loader.annotate_clusterids(fld_in=False, 
                                              fld_out = 'clusterid')
    datasets = loader.annotate(datasets,annotate_function = clusteranno)
    annotate_function = loader.annotate_celltypes(fld_in='clusterid',
                                                  fld_out = 'celltype',
                                                  path = path)
    datasets = loader.annotate(datasets, annotate_function= annotate_function)
    loader.save(datasets, format = 'h5')
'''

In [ ]:
datasets = loader.read(dir = path, 
                       datasets = ds,
                       suffix = '.h5',
                       sample_size = 1000, 
                       remove_cells = {'celltype': "no pangalo"})
#TODO map (sc.pp.filter(mingenes)), dont put mingenes into the reader (as it its also used as csv)


In [ ]:
%%time 
# was "neighbors"
from cellsaw import similarity as sim
target_datasets = datasets[:2] # the one we want to annotate
source_datasets = datasets[2:10] # the database

# TODO rank_by_similarity needs a doc string,
ranked_datasets_list, similarity_df = sim.rank_by_similarity(
                                    target = target_datasets, 
                                    source = source_datasets,            
                                    return_similarity = True)



In [ ]:
sim.plot_dendrogram(similarity_df)
# turn dendrogram, switch places with matrix



In [ ]:
%%time 

from cellsaw import merge 
id = 0
target = target_datasets[id]
source = ranked_datasets_list[id][0]

target = merge.annotate_label(target,source,source_label = 'celltype', 
                               target_label='predicted_celltype', 
                               pca_dim = 20, umap_dim = None,
                       n_intra_neighbors = 7,
                       n_inter_neighbors = 2,
                       similarity_scale_factor = 1.0)

In [ ]:
from cellsaw import sim


'''


id = 0
target = target_datasets[id]
source = ranked_datasets_list[id][0]

target = annotate_label(target,source,source_label = 'celltype', 
                               target_label='predicted_celltype', 
                               pca_dim = 20, umap_dim = None)

                
                


target = annotate_label_linsum_copylabel(source_label = 'celltype', 
                               target_label= 'diffuseknn_celltype',
                               pca_dim = 20, umap_dim = None)

# just stack the data and do knn
target = annotate_label_knn(target,source,source_label = 'celltype', 
                               target_label='', 
                               pca_dim = 20, umap_dim = None)

# diffusion on stacked data
target = annotate_label_raw_diffusion(target,source,source_label = 'celltype', 
                               target_label='', 
                               pca_dim = 20, umap_dim = None)


target = markercount(target,source,source_label = 'celltype', 
                               target_label='markercount_celltype', 
                               pca_dim = 20, umap_dim = None)



plot(source, target, source_label = 'celltype',target_label = 'predicted_celltype')
# cmp3 with the overlap plot 

# add more lines to 

acc = accuracy_evaluation(target,true='celltype', predicted = 'predicted_celltype') 

# make a dataframe, where i loop over the targets (ID above) and accumulate accuracies, columns for hyperparameters




# iterate over neighbors, weight by dataset distance, to obtain weighted avg prediction for cells

# TODO: distance multiplier for linear_sum_assignment
# avg 1nn dist: scale that (default1)
# TODO: build a generic annotator for cells 


'''


In [ ]:
z= list('abcde')

In [ ]:
z

In [ ]:
args = [iter(z)] * 3


In [ ]:
zip(*args)

In [ ]:
import cellsaw.merge.hungarianEM as h

In [ ]:
from sklearn.datasets import make_blobs

samples = [20, 80, 20]
X, y = make_blobs(n_samples=samples, centers=None, n_features=2,
              random_state=0)

size  = sum(samples)//3


from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances as ed
import numpy as np
from ubergauss import tools as t


def assign(X,y):
    centers = h.centers(y[y!= -1], X[y!=-1])
    distC = ed(X, centers)
    y = np.argmin(distC,axis = 1)
    return y, distC



def unassign(distC,y):

    # for each cluster 
    for i in np.unique(y):
        # if it is too large:
        if sum(y==i) > size:
            
            

            
            if False:
                distances = distC[y==i]
                closest = np.argsort(distances, axis = 1)
                dist = [ distances[i,row[1]] - distances[i,row[0]] for i,row in enumerate(closest)]
                dist = np.array(dist)
                remove = t.binarize(-dist,sum(y==i)-size).astype(bool)
                print(f"{dist}{remove}")
                indi = np.full(sum(y==i),i)
                indi[remove] = -1
                y[y==i] = indi
            
            distances = distC[y==i]
            closest = np.argsort(distances, axis = 1)
            items = [ (distances[i,row[0]] , distances[i,row[1]] , row[0],row[1]) for i,row in enumerate(closest)]
            items.sort(key = lambda x: x[1]/x[0])
            
            y[y==i] = np.array([ e[2] if z < size else e[3]  for z,e in enumerate(items)])
            
            

            
    return y



y, distC = assign(X,y)
for i in range(10):
    '''
    draw
    unassign 
    draw 
    reassign 
    draw
    show
    '''
    
    plt.scatter(X[:,0],X[:,1], c = y, s = 200)
    
    y = unassign(distC,y)
    
    pX = X[y==-1]
    print(pX)
    plt.scatter(pX[:,0],pX[:,1], c = 'r', s=150)
    
    y, distC = assign(X,y)
    
    plt.scatter(X[:,0],X[:,1], c = y, s=100)
    
    plt.show()
    for z in np.unique(y):
        print(f"{z}: {sum(y==z)}")

In [ ]:
z = np.array([1,2,3,4,4,4])
z[z==4] = 1
print(z)